# Sectors Review

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [30]:
data = pd.read_csv('../../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740.0,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,"3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US",...,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08,2024-06-12,2024-05-23
AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142.0,NYSE,USD,USA,MANUFACTURING,HOUSEHOLD APPLIANCES,"11270 WEST PARK PLACE, MILWAUKEE, WI, US",...,3.248,15.19,1.141,92.44,63.34,83.92,81.37,1.207840e+08,2024-08-15,2024-07-31
ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"100 ABBOTT PARK ROAD, ABBOTT PARK, IL, US",...,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09,2024-08-15,2024-07-15
ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"1 NORTH WAUKEGAN ROAD, NORTH CHICAGO, IL, US",...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09,2024-08-15,2024-07-15
ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373.0,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",...,3.041,18.21,1.227,384.33,277.47,301.44,329.36,6.263840e+08,2024-08-15,2024-07-11


In [31]:
data.drop(['AssetType',
           'CIK',
           'Description',
           'Currency',
           'Country',
           'Address',
           'DividendPerShare',
           'FiscalYearEnd',
           'LatestQuarter',
           'DividendDate',
           'ExDividendDate',
          ], inplace=True, axis=1)

In [32]:
data.replace('None', np.nan, inplace=True)

In [33]:
data['MarketCapitalization'] = data['MarketCapitalization'].astype(float)
data['Beta'] = data['Beta'].astype(float)
data['PEGRatio'] = data['PEGRatio'].astype(float)
data['PERatio'] = data['PERatio'].astype(float)
data['BookValue'] = data['BookValue'].astype(float)
data['DividendYield'] = data['DividendYield'].astype(float)
data['EPS'] = data['EPS'].astype(float)
data['RevenuePerShareTTM'] = data['RevenuePerShareTTM'].astype(float)
data['ProfitMargin'] = data['ProfitMargin'].astype(float)
data['OperatingMarginTTM'] = data['OperatingMarginTTM'].astype(float)
data['ReturnOnAssetsTTM'] = data['ReturnOnAssetsTTM'].astype(float)
data['EBITDA'] = data['EBITDA'].astype(float)
data['RevenueTTM'] = data['RevenueTTM'].astype(float)
data['GrossProfitTTM'] = data['GrossProfitTTM'].astype(float)
data['DilutedEPSTTM'] = data['DilutedEPSTTM'].astype(float)
data['QuarterlyEarningsGrowthYOY'] = data['QuarterlyEarningsGrowthYOY'].astype(float)
data['QuarterlyRevenueGrowthYOY'] = data['QuarterlyRevenueGrowthYOY'].astype(float)

In [34]:
data['Sector'].unique()

array(['LIFE SCIENCES', 'MANUFACTURING', 'TRADE & SERVICES', 'TECHNOLOGY',
       'ENERGY & TRANSPORTATION', 'FINANCE', 'REAL ESTATE & CONSTRUCTION'],
      dtype=object)

In [35]:
data.groupby(['Sector']).count()['Name'].to_frame().sort_values('Name', ascending=False)

,Name
Sector,
MANUFACTURING,96
LIFE SCIENCES,80
ENERGY & TRANSPORTATION,76
TRADE & SERVICES,75
TECHNOLOGY,69
FINANCE,65
REAL ESTATE & CONSTRUCTION,42


## LIFE SCIENCES

In [36]:
lf = data[data['Sector']=='LIFE SCIENCES'].copy()
lf.drop('Sector', axis=1, inplace=True)

In [37]:
lf.head()

,Name,Exchange,Industry,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendYield,EPS,...,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,3M Company,NYSE,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,5.808631e+10,8.146000e+09,NaN,1.905,8.790,0.0267,-12.73,...,1.546,10.51,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08
ABT,Abbott Laboratories,NYSE,PHARMACEUTICAL PREPARATIONS,1.804344e+11,1.048500e+10,32.72,5.990,22.310,0.0212,3.17,...,4.72,4.904,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09
ABBV,AbbVie Inc,NYSE,PHARMACEUTICAL PREPARATIONS,3.065727e+11,2.611600e+10,51.52,0.449,4.535,0.0357,3.37,...,5.29,27.74,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09
A,Agilent Technologies Inc,NYSE,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,3.892675e+10,1.612000e+09,31.62,2.701,21.310,0.0071,4.22,...,6.12,6.66,6.24,24.88,1.049,155.06,96.30,136.10,133.04,2.917610e+08
APD,Air Products and Chemicals Inc,NYSE,INDUSTRIAL INORGANIC CHEMICALS,5.525416e+10,4.115100e+09,22.45,1.372,67.080,0.0269,11.07,...,4.143,3.461,4.989,13.79,0.822,299.60,209.25,263.88,257.75,2.223060e+08


In [38]:
lf.sort_values('MarketCapitalization', ascending=False).head(10)

,Name,Exchange,Industry,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendYield,EPS,...,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding
Symbol,,,,,,,,,,,,,,,,,,,,,
LLY,Eli Lilly and Company,NYSE,PHARMACEUTICAL PREPARATIONS,7.243399e+11,1.337370e+10,117.78,1.3040,14.230,0.0062,6.83,...,20.16,56.54,20.82,77.19,0.409,966.10,447.24,867.06,726.83,9.004050e+08
PG,Procter & Gamble Company,NYSE,"SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUME...",3.962049e+11,2.288500e+10,27.92,3.5560,21.450,0.0240,6.02,...,4.715,7.99,5.0,18.61,0.400,170.88,137.96,166.29,158.10,2.357100e+09
JNJ,Johnson & Johnson,NYSE,PHARMACEUTICAL PREPARATIONS,3.869879e+11,3.118100e+10,24.32,0.8880,29.720,0.0309,6.61,...,4.174,5.17,4.25,15.52,0.517,170.63,141.57,149.71,153.44,2.407240e+09
ABBV,AbbVie Inc,NYSE,PHARMACEUTICAL PREPARATIONS,3.065727e+11,2.611600e+10,51.52,0.4490,4.535,0.0357,3.37,...,5.29,27.74,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09
MRK,Merck & Company Inc,NYSE,PHARMACEUTICAL PREPARATIONS,2.917797e+11,3.305500e+10,21.33,0.0962,15.940,0.0267,5.40,...,4.67,7.23,5.13,19.91,0.389,134.63,97.23,126.93,120.11,2.532810e+09
TMO,Thermo Fisher Scientific Inc,NYSE,"MEASURING & CONTROLLING DEVICES, NEC",2.292129e+11,1.081900e+10,37.29,2.4780,124.170,0.0027,16.09,...,5.41,4.832,6.04,21.97,0.771,622.30,414.76,568.09,547.03,3.819960e+08
LIN,Linde plc Ordinary Shares,NASDAQ,INDUSTRIAL INORGANIC CHEMICALS,2.182451e+11,1.220200e+10,35.28,2.7970,80.780,0.0123,12.87,...,6.66,5.68,7.12,18.61,0.944,476.18,357.70,439.22,427.34,4.807160e+08
DHR,Danaher Corporation,NYSE,"INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLA...",1.978214e+11,7.460000e+09,50.26,3.1270,68.920,0.0039,5.45,...,7.44,3.322,8.01,25.53,0.861,277.72,181.51,255.82,238.28,7.222130e+08
ABT,Abbott Laboratories,NYSE,PHARMACEUTICAL PREPARATIONS,1.804344e+11,1.048500e+10,32.72,5.9900,22.310,0.0212,3.17,...,4.72,4.904,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09


In [39]:
lf.sort_values('Beta', ascending=False).head(10)

,Name,Exchange,Industry,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendYield,EPS,...,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding
Symbol,,,,,,,,,,,,,,,,,,,,,
HCA,HCA Holdings Inc,NYSE,"SERVICES-GENERAL MEDICAL & SURGICAL HOSPITALS,...",9.369376e+10,1.335600e+10,17.04,1.442,-6.17,0.0073,21.31,...,1.314,293.44,1.944,9.93,1.688,366.10,214.61,332.10,298.75,258074000.0
MRNA,Moderna Inc,NASDAQ,"BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)",3.328101e+10,-4.107000e+09,NaN,0.000,30.50,NaN,-15.35,...,6.59,2.842,5.22,4.641,1.662,170.47,62.55,131.38,105.61,384396000.0
ALGN,Align Technology Inc,NASDAQ,"ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES &...",1.876848e+10,8.167840e+08,41.14,1.962,49.94,NaN,6.06,...,6.14,6.53,5.92,28.6,1.659,413.20,176.34,252.26,266.91,75282000.0
ALB,Albemarle Corp,NYSE,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...",1.099700e+10,-3.170970e+08,33.90,1.508,79.11,0.0173,2.76,...,1.498,1.531,1.851,22.45,1.575,213.26,90.02,110.11,123.88,117527000.0
TER,Teradyne Inc,NASDAQ,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,2.005107e+10,6.281790e+08,40.42,1.683,16.88,0.0042,3.04,...,7.42,6.82,6.75,26.26,1.499,163.21,80.81,142.67,114.05,163176000.0
TRMB,Trimble Inc,NASDAQ,"MEASURING & CONTROLLING DEVICES, NEC",1.261334e+10,7.663000e+08,8.45,1.885,22.98,NaN,6.11,...,3.396,2.239,3.654,5.67,1.498,65.55,39.57,55.14,54.47,244208000.0
EMN,Eastman Chemical Company,NYSE,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...",1.184575e+10,1.574000e+09,13.56,1.141,48.17,0.0322,7.43,...,1.251,2.111,1.718,8.74,1.474,105.12,67.12,98.97,90.20,116810000.0
MOS,The Mosaic Company,NYSE,AGRICULTURAL CHEMICALS,8.799740e+09,2.077600e+09,11.60,1.831,37.40,0.0307,2.36,...,0.689,0.732,1.037,6.68,1.416,41.76,26.54,28.90,31.83,321393000.0
ISRG,Intuitive Surgical Inc,NASDAQ,"ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES &...",1.602433e+11,2.386000e+09,77.48,7.130,41.40,NaN,5.82,...,19.15,10.25,18.41,59.92,1.390,468.78,254.85,429.76,371.07,355354000.0


In [14]:
lf.groupby('Industry').agg({'Name': 'count',
                            'MarketCapitalization': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                           })

Name MarketCapitalization  \
                                                   count                  sum   
Industry                                                                        
AGRICULTURAL CHEMICALS                                 2         2.223604e+10   
AGRICULTURAL PRODUCTION-CROPS                          1         3.860550e+10   
AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMM...     1         7.950805e+10   
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)        5         3.544207e+11   
CHEMICALS & ALLIED PRODUCTS                            1         8.127291e+09   
ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS          1         1.045178e+11   
IN VITRO & IN VIVO DIAGNOSTIC SUBSTANCES               1         3.932146e+10   
INDUSTRIAL INORGANIC CHEMICALS                         2         2.734992e+11   
INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY...     5         3.442476e+11   
INDUSTRIAL ORGANIC CHEMICALS                           2         5.597436e+10   
INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY &...     3         8.519576e+10   
LABORATORY ANALYTICAL INSTRUMENTS                      4         7.335886e+10   
MEASURING & CONTROLLING DEVICES, NEC                   3         2.702312e+11   
OPHTHALMIC GOODS                                       1         1.855998e+10   
OPTICAL INSTRUMENTS & LENSES                           1         9.373232e+10   
ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES & ...     5         2.622998e+11   
PAINTS, VARNISHES, LACQUERS, ENAMELS & ALLIED P...     1         2.785369e+10   
PERFUMES, COSMETICS & OTHER TOILET PREPARATIONS        3         1.533251e+11   
PHARMACEUTICAL PREPARATIONS                           12         2.497098e+12   
PLASTIC MATERIAL, SYNTH RESIN/RUBBER, CELLULOS ...     1         1.495222e+10   
PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN ELA...     4         9.341702e+10   
SERVICES-GENERAL MEDICAL & SURGICAL HOSPITALS, NEC     2         1.082639e+11   
SERVICES-MEDICAL LABORATORIES                          2         3.604424e+10   
SERVICES-MISC HEALTH & ALLIED SERVICES, NEC            1         1.205121e+10   
SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUMES...     3         4.910279e+11   
SPECIALTY CLEANING, POLISHING AND SANITATION PR...     1         1.671695e+10   
SURGICAL & MEDICAL INSTRUMENTS & APPARATUS            10         4.753832e+11   
X-RAY APPARATUS & TUBES & RELATED IRRADIATION A...     2         5.612110e+10   

                                                                         \
                                                             min idxmin   
Industry                                                                  
AGRICULTURAL CHEMICALS                              8.799740e+09    MOS   
AGRICULTURAL PRODUCTION-CROPS                       3.860550e+10   CTVA   
AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMM...  7.950805e+10     TT   
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)     1.223017e+10   TECH   
CHEMICALS & ALLIED PRODUCTS                         8.127291e+09    FMC   
ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS       1.045178e+11    MDT   
IN VITRO & IN VIVO DIAGNOSTIC SUBSTANCES            3.932146e+10   IDXX   
INDUSTRIAL INORGANIC CHEMICALS                      5.525416e+10    APD   
INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY...  2.494786e+10    FTV   
INDUSTRIAL ORGANIC CHEMICALS                        2.523889e+10    IFF   
INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY &...  2.005107e+10    TER   
LABORATORY ANALYTICAL INSTRUMENTS                   8.760474e+09    BIO   
MEASURING & CONTROLLING DEVICES, NEC                1.261334e+10   TRMB   
OPHTHALMIC GOODS                                    1.855998e+10    COO   
OPTICAL INSTRUMENTS & LENSES                        9.373232e+10   KLAC   
ORTHOPEDIC, PROSTHETIC & SURGICAL APPLIANCES & ...  1.876848e+10   ALGN   
PAINTS, VARNISHES, LACQUERS, ENAMELS & ALLIED P...  2.785369e+10    PPG   
PERFUMES, COSMETICS & OTHER TOILET PREPA

In [12]:
summ = lf.groupby('Industry').agg({'Name': 'count',
                            'MarketCapitalization': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'EBITDA': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'BookValue': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'DividendYield': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'EPS': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'RevenuePerShareTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'ProfitMargin': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'OperatingMarginTTM': ['min', 'idxmin', 'max', 'idxmax'],
                            'ReturnOnAssetsTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'ReturnOnEquityTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'RevenueTTM': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'QuarterlyEarningsGrowthYOY': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                            'QuarterlyRevenueGrowthYOY': ['sum', 'min', 'idxmin', 'max', 'idxmax'],
                           })
summ_pv = pd.DataFrame(columns=summ.index, index=summ.columns, data=summ.to_numpy().transpose())
summ_pv

Industry                         AGRICULTURAL CHEMICALS  \
Name                      count                       2   
MarketCapitalization      sum             22236045000.0   
                          min              8799740000.0   
                          idxmin                    MOS   
                          max             13436305000.0   
...                                                 ...   
QuarterlyRevenueGrowthYOY sum                    -0.526   
                          min                    -0.269   
                          idxmin                     CF   
                          max                    -0.257   
                          idxmax                    MOS   

Industry                         AGRICULTURAL PRODUCTION-CROPS  \
Name                      count                              1   
MarketCapitalization      sum                    38605500000.0   
                          min                    38605500000.0   
                          idxmin                          CTVA   
                          max                    38605500000.0   
...                                                        ...   
QuarterlyRevenueGrowthYOY sum                            -0.08   
                          min                            -0.08   
                          idxmin                          CTVA   
                          max                            -0.08   
                          idxmax                          CTVA   

Industry                         AUTO CONTROLS FOR REGULATING RESIDENTIAL & COMML ENVIRONMENTS  \
Name                      count                                                   1              
MarketCapitalization      sum                                         79508054000.0              
                          min                                         79508054000.0              
                          idxmin                                                 TT              
                          max                                         79508054000.0              
...                                                                             ...              
QuarterlyRevenueGrowthYOY sum                                                 0.128              
                          min                                                 0.128              
                          idxmin                                                 TT              
                          max                                                 0.128              
                          idxmax                                                 TT              

Industry                         BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)  \
Name                      count                                                5   
MarketCapitalization      sum                                     354420660000.0   
                          min                                      12230172000.0   
                          idxmin                                            TECH   
                          max                                     178761597000.0   
...                                                                          ...   
QuarterlyRevenueGrowthYOY sum                                             -0.064   
                          min                                             -0.299   
                          idxmin                                            MRNA   
                          max                                               0.22   
                          idxmax                                            AMGN   

Industry                         CHEMICALS & ALLIED PRODUCTS  \
Name                      count                            1   
MarketCapitalization      sum                   8127291000.0   
                          min                   8127291000.0   
                          idxmin                        